# Capstone project notebook

In [1]:
# Importing all my modules
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Scraping the wikipedia page

In [2]:
# scraping the wikipedia page
from pandas.io.html import read_html
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

wikitables = read_html(page,  attrs={"class":"wikitable sortable"})

print ("Extracted {num} wikitables".format(num=len(wikitables)))

df = wikitables[0]
print(df.head())


Extracted 1 wikitables
  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront


## Creating the required DataFrame

In [3]:
df = df.loc[(df.Borough != 'Not assigned')] # Removing the not assigned values in Borough

df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(",".join).reset_index() # Grouping the Neighbourhood with same Postcode

df.Neighbourhood[df.Neighbourhood == 'Not assigned'] = df.Borough # Replacing the Not Assigned Neighbourhood with Borough name

df = pd.DataFrame(df)
print(df.head(10))

print(df.columns)

  Postcode      Borough                                  Neighbourhood
0      M1B  Scarborough                                  Rouge,Malvern
1      M1C  Scarborough           Highland Creek,Rouge Hill,Port Union
2      M1E  Scarborough                Guildwood,Morningside,West Hill
3      M1G  Scarborough                                         Woburn
4      M1H  Scarborough                                      Cedarbrae
5      M1J  Scarborough                            Scarborough Village
6      M1K  Scarborough      East Birchmount Park,Ionview,Kennedy Park
7      M1L  Scarborough                  Clairlea,Golden Mile,Oakridge
8      M1M  Scarborough  Cliffcrest,Cliffside,Scarborough Village West
9      M1N  Scarborough                     Birch Cliff,Cliffside West
Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')


In [4]:
# Number of rows and columns in dataframe
print(df.shape)

(103, 3)
